In [32]:
import requests
import logging
import urllib
import json
import ndjson
import pandas as pd
import sqlite3
from Extractor import *

LOG = logging.getLogger(__name__)
API_URL = 'https://lichess.org/'
    

In [2]:
API_URL = 'https://lichess.org/'
AccessToken = "lip_FdeTEN7DhU3ge3Eg8Shu"
API = LichessRequestor(Token = AccessToken, base_url = API_URL)

In [4]:
path = f'api/team/agadmators-team/users'
data = API.get(path=path)

In [5]:
users_raw = LichessRequestor.parse(data)

In [6]:
user_list = LichessRequestor.parse_team(users_raw)

In [ ]:
## Create the sqlite3 connection to db
con = sqlite3.connect('chessdb.db')
cursor = con.cursor()

In [7]:
user_list.to_csv("user_list.csv")
## Use pandas to create the new table in the database
user_raw.to_sql("users", con, if_exists = "replace")

,id,Blitz_Games,Blitz_rating,Bullet_Games,Bullet_rating,Rapid_Games,Rapid_rating,country
0,mgm2890,0,1500,1,1408,338,2014,No Country
1,sajjad_fathi,664,1717,13,1137,93,1790,IR
2,cagribgoode,1,1702,0,1500,8,1591,No Country
3,conanzhan,32,1214,59,1052,19,1019,GB-ENG
4,koroshkabir1,211,2008,8,1339,0,1500,No Country
...,...,...,...,...,...,...,...,...
20928,rvgupta1996,3759,2080,981,1905,1534,1969,IN
20929,jahjgv,20,1783,5,1470,7,1804,No Country
20930,suhail59,146,1477,6784,1497,72,1706,No Country
20931,hasidicwilliamsburg,7631,2130,3338,2003,1032,2097,No Country


In [ ]:
## Use pandas to upload country table lookup
## file from: https://stefangabos.github.io/world_countries/
countries = pd.read_csv("countries.csv")
countries.to_sql("country", con, if_exists = "replace")

In [ ]:
## Query to clean the data
query = """
Select t1.id, t1.blitz_games, t1.blitz_rating, t1.rapid_games, t1.rapid_rating, t1.bullet_games, 
t1.bullet_rating, t1.country
FROM
(SELECT 
    id, blitz_games, blitz_rating, rapid_games, rapid_rating, bullet_games, bullet_rating,
    (CASE country
        WHEN "GB-ENG" THEN "UK"
        WHEN "GB-NIR" THEN "UK"
        WHEN "GB-SCT" THEN "UK"
        WHEN "GB-WLS" THEN "UK"
        WHEN "_united-nations" THEN "UK"
        WHEN "ES-CT" THEN "ES"
        WHEN "CA-QC" THEN "CA"
        ELSE country
    END) as country
FROM users 
where 
country not in ("No Country", "_east-turkestan", "_adygea", "_earth", "_belarus-wrw", 
"_lichess", "_pirate", "_rainbow", "east-turkestan")
AND blitz_games > 100
AND rapid_games > 100
AND bullet_games > 100) t1

"""
result = cursor.execute(query)
## Getting the list of all users
users = pd.read_sql_query(query, con)

In [ ]:
## Random sampling from the user dataset
## Querying all the data would take 60 hours...
userlist_cleaned = []
AllUsers = pd.DataFrame()
Ratingless800 = users[users['blitz_rating'] <= 800]
Rating800to1000 = users[(users['blitz_rating'] > 800) & (users['blitz_rating'] <= 1000)]
Rating1000to1200 = users[(users['blitz_rating'] > 1000) & (users['blitz_rating'] <= 1200)].sample(100)
Rating1200to1400 = users[(users['blitz_rating'] > 1000) & (users['blitz_rating'] <= 1200)].sample(100)
Rating1400to1600 = users[(users['blitz_rating'] > 1400) & (users['blitz_rating'] <= 1600)].sample(250)
Rating1600to1800 = users[(users['blitz_rating'] > 1600) & (users['blitz_rating'] <= 1800)].sample(250)
Rating1800to2000 = users[(users['blitz_rating'] > 1800) & (users['blitz_rating'] <= 2000)].sample(250)
Rating2000to2200 = users[(users['blitz_rating'] > 2000) & (users['blitz_rating'] <= 2200)].sample(250)
RatingAbove2200 = users[(users['blitz_rating'] > 2200)].sample(250)
userlist_cleaned = [Ratingless800, Rating800to1000, Rating1000to1200, Rating1200to1400, Rating1200to1400
        ,Rating1400to1600, Rating1600to1800, Rating1800to2000, Rating2000to2200, RatingAbove2200]
UsersSelected = pd.concat(userlist_cleaned, ignore_index = True)

In [ ]:
UsersSelected.to_csv("Users_Selected.csv")

In [ ]:
## Query games from user database
all_games = []
all_games_df = pd.DataFrame()
for i in range(len(UsersSelected)):
    params = {
        'max': 15,
        'rated': True,
        'perfType': 'blitz,rapid,bullet',
        'analysed': True,
        'evals': True,
        'opening': True,     
    }
    username = UsersSelected['id'][i]
    path = f'api/games/user/{username}'
    games = API.get(path=path, params = params)
    games_data = LichessRequestor.parse(games, convert=False)
    data = LichessRequestor.parse_games(games_data, LichessRequestor)
    all_games.append(data)
all_games_df = pd.concat(all_games, ignore_index = True)

In [ ]:
all_games_df.to_csv("all_games.csv")

In [ ]:
all_games_dict = all_games_df.to_dict()
all_games_df_with_countries = LichessRequestor.parse_countries(all_games_dict)

In [ ]:
all_games_df_with_countries['black_country'].replace({"GB-ENG": "UK", "GB-NIR": "UK", "GB-SCT": "UK", 
                                                      "GB-WLS": "UK", "_united-nations" : "UN", "ES-CT": "ES", 
                                                      "CA-QC": "CA"}, inplace = True)
all_games_df_with_countries['white_country'].replace({"GB-ENG": "UK", "GB-NIR": "UK", "GB-SCT": "UK", 
                                                      "GB-WLS": "UK", "_united-nations" : "UN", "ES-CT": "ES", 
                                                      "CA-QC": "CA"}, inplace = True)

In [ ]:
all_games_df_with_countries.to_csv("games_with_country.csv")

In [ ]:
games_cleaned = all_games_df_with_countries[(all_games_df_with_countries['white_country'].str.len() == 2) 
                & (all_games_df_with_countries['black_country'].str.len() == 2)]
games_cleaned.reset_index(inplace = True, drop = True)

In [ ]:
games_cleaned_w_country = games_cleaned.to_dict()
all_games_final = LichessRequestor.parse_winstreak(games_cleaned_w_country)

In [ ]:
all_games_final.to_csv("All Games Final.csv")